# 1 - Simple functions

## 1.1 - Introduction

[```Pythas```](https://github.com/pinselimo/Pythas) is a Python package that can handle all the manual work to bind Haskell Modules to Python. To get you started in using Pythas, it provides an ```Examples.hs``` file inside the ```examples``` directory. Here we'll have a closer look at this file. If you don't have it, it also comes with this repo.

## 1.2 - The Examples.hs file

The [```Examples.hs```](https://github.com/pinselimo/Pythas/blob/master/example/Example.hs) file is up to date with the features Pythas provides. It really is supposed to be a showcase and a quick reference of what's possible. So if you find a type annotation for a function in it, you can be assured that these types are in fact supported by Pythas.

To use it, we first have to import ```pythas```:

In [1]:
import pythas

Compiling with: stack


Now we can import the Haskell module we placed in a directory relative to the location of this notebook at: ```examples/Examples.hs```

In [2]:
import example.example as e

Compiling with: stack


### 1.2.1 - Hello Haskell

As tradition demands, our first example with be a simple 'Hello World' function. So right off the bat we're already in the unsanitory land of impure functions. Probably not the reason why you're interested in using Haskell from Python, but bear with me.

This is how the function looks in the bare Haskell module file:

~~~haskell
hello :: IO ()
hello = putStrLn "Hello from Haskell!"
~~~

**Important:** Note the top level type declaration. ```Pythas``` relies on it to gain information about your functions. Any function without a type declaration will be ignored (which is on purpose can be put to good use of course). Moreover, any function that cannot be exported will also be ignored (e.g. ```someComplicatedFunc``` in ```Example.hs```)

We can now use this function from Python, as simple as if it were Python code all along:

In [3]:
e.hello()

```Hello from Haskell!``` < it says in the console

### 1.2.2 - Pure Functions

Ok, that wasn't extremely useful inside a Jupyter Notebook. But we're all here for the purity, so let's delve into the things we cannot have in Python.
Our ```Example.hs``` file defines various pure functions. One of which is ```square``` which squares an integer:

~~~haskell
square :: CInt -> CInt
square i = i * i
~~~

Note: It could as well be typed as ```Int -> Int``` or ```Ìnteger -> Integer``` that's all supported by Pythas!
Can we use this function from Python now?

In [4]:
e.square(4)

16

What about memory limits, are we safe against overflows? - Nope ;)

In [5]:
e.square(2**16)

0

Note: While ```Integer``` types are only limited by memory in the Haskell RTS, they are translated to ```llong``` by Pythas.

You can also mix types and use ```Double```s in your code, as shown by the somewhat stupid ```multisin``` function:

~~~haskell
multisin :: Int -> Double -> Double
multisin x y = (fromIntegral x) * (sin y)
~~~

Let's try just to see what happens:

In [6]:
pi = 355/113 # close enough

e.multisin(63, 2*pi)

3.361228784959365e-05

### 1.2.3 - Lists

A big field of application for Python is of course Data Science. So it would make sense to shove a large amount of data over to Haskell all at once, right?

Pythas supports lists as input and/or output types of Haskell functions. Also lists of lists, and lists of lists of lists and so on ...

So e.g. you wanted to have a Haskell function that calculates the quarter of every integer in a list to return a list of doubles:

~~~haskell
mapQuarter :: [Integer] -> [Double]
mapQuarter = map ((*0.25) . fromIntegral)
~~~

How would you call it in Python?

In [7]:
e.mapQuarter(range(10))

[0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25]

In [8]:
e.mapQuarter([x for x in range(10)])

[0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25]

Any iterable type really is enough. Keep track of the types inside the iterable though, don't go too much off track, Haskell expects a list of integers nontheless:

In [9]:
# e.mapQuarter((10,4.5)) # -> TypeError

As mentioned, nested lists are no problem. Currently the only example using it is an identity function for lists of lists of strings:

In [10]:
e.nested(['Haskell' if i%2 else 'Python' for i in range(2)]*2)

[['P', 'y', 't', 'h', 'o', 'n'],
 ['H', 'a', 's', 'k', 'e', 'l', 'l'],
 ['P', 'y', 't', 'h', 'o', 'n'],
 ['H', 'a', 's', 'k', 'e', 'l', 'l']]

Which brings us to our next topic:

### 1.2.4 - Strings

Strings are lists of chars in Haskell. But Pythas' Python backend ```ctypes``` doesn't quite treat them fairly. Also strings cannot be "nested", we already know a char is not a list and they are NUL terminated. All good reasons to treat them a little bit differently as well. 

As a user of Pythas you only see the example above as a reminder of the different perception of strings comparing Haskell to Python. Otherwise, strings are just strings. The ```Example.hs``` file provides a ```strings``` function, which filters out any ```Char``` matching ```'a'```. How does it work in Python?

In [11]:
e.strings("Haskell")

'Hskell'

### 1.2.5 - Tuples

That's all nice, but what if I have a collection which consists of elements with differing types? Tuples to the rescue!

In our ```Examples.hs``` file we find one example of a tuple as return type:

~~~haskell
tuple :: (CInt, String)
tuple = (1 :: CInt, "a")
~~~

So let's try it out:

In [12]:
e.tuple()

(1, 'a')

Tuples can also be used as arguments! A simple example is a ```snd``` "equivalent" ```hsnd``` which only works on ```(Int, Int)``` (dunno what that for ;) - let's just have a look:

In [13]:
e.hsnd((42,63))

63

### 1.2.6 - About Constants

Why did we have to call ```e.tuple``` if it obviously is a constant?

Well let's say the ```()``` you have to add in Python is there to remind you something's going on in the ```IO``` monad. See, tuples cannot be passed by value in between Haskell and Python. So Pythas has to allocate memory on the heap, which obviously can only be done inside of ```IO```.

Thus, the last important thing to note is: Constants have to be called if they are non-trivial types! (So: Strings, Lists, Tuples)

In [14]:
e.someConstant # Integer works fine

63

In [15]:
e.haskellList() # List needs allocation

[63]

And that's it. Lists and tuples can be nested as wished in the output btw!